**Import necessari**

In [ ]:
import keras 
from keras.layers.core import Dense, Activation
from keras.optimizers import SGD
from keras.layers import Input
from keras import models
from keras.models import Model
import numpy as np
import tensorflow as tf

**Funzioni utilizzate**

In [ ]:
def polar_generator(batchsize,grid=(10,10),noise=.002,flat=False):
  while True:
    x = np.random.rand(batchsize)
    y = np.random.rand(batchsize)
    out = np.zeros((batchsize,grid[0],grid[1]))
    xc = (x*grid[0]).astype(int)
    yc = (y*grid[1]).astype(int)
    for b in range(batchsize):
      out[b,xc[b],yc[b]] = 1
    #compute rho and theta and add some noise
    rho = np.sqrt(x**2+y**2) + np.random.normal(scale=noise)
    theta = np.arctan(y/np.maximum(x,.00001)) + np.random.normal(scale=noise)
    if flat:
      out = np.reshape(out,(batchsize,grid[0]*grid[1]))
    yield ((theta,rho),out)

def my_accuracy_wrapper(real,predict):
  # accuracy = true positive / (true positive + true negative)
  np_r = real.numpy()
  np_p = predict.numpy()
  # cerco il massimo in entrambi ignorando quindi gli altri numeri 
  # i quali dovrebbe essere tutti zero per "real"
  max_r = np.argmax(np_r, axis=1)
  max_p = np.argmax(np_p, axis=1)
  max_e = np.equal(max_r,max_p)
  # questo somma solo gli 1 ovvero il numero dei positivi
  true_positive = np.sum(max_e)
  true_positive_true_negative = len(max_e)
  return true_positive / true_positive_true_negative

def my_accuracy(real, predict):
  # Codice sorgente della categorical_accuracy di keras: 
  # K.mean(K.equal(K.argmax(y_true, axis=-1), K.argmax(y_pred, axis=-1)))
  # Converto i tensori in formati che numpy e' in grado di elaborare
  return tf.py_function(
      func=my_accuracy_wrapper,
      inp=[real, predict],
      Tout=tf.float32
    )



**Definizione del modello**

In [ ]:
# la scelta di usare due input separati e' dovuta al ridotto numero di input che 
# abbiamo a dispozione e anche perche' i due input non sappiamo se hanno o meno
# una relazione tra loro quindi decidiamo di considerarli almeno nella prima
# fase in maniera indipendente
theta = Input(shape=(1))
rho = Input(shape=(1))
# softsign e tanh sono state scelta in maniera sperimentale
# provando per ordine: 
# (swish, relu), (tanh, relu), (swish, tanh), (sofsign, swish), (softsign, tanh)
dense_1 = Dense(2, activation='softsign')(theta)
dense_2 = Dense(2, activation='tanh')(rho)
# inception module
mid_1 = tf.keras.layers.concatenate([dense_1, dense_2], axis = 1)

# elu sembra essere una versione migliorata della relu
# la relu risulta una buona scelta nelle reti profonde
elu_1= Dense(4, activation='elu')(mid_1)
elu_2 = Dense(4, activation='elu')(elu_1)
# la softmax permette di distribuire la probabilita'
output = Dense(100, activation='softmax')(elu_2)

model = Model([theta,rho], output)

model.summary()

**Fase di training del modello**

In [ ]:
n_train = 3000000

g1,g2 = 10,10
gen = polar_generator(n_train,grid=(g1,g2),noise=0.002,flat=True)

# epoche e batch size sono stati scelti abbastanza alti perche' nella prima fase
# lo scopo e' quello di aumentare quanto possibile l'accuracy

epochs = 150
batch_size = 2048

opt = keras.optimizers.Adam()
model.compile(optimizer=opt, loss='categorical_crossentropy', 
        metrics=['accuracy', my_accuracy]
              )

(theta,rho),y = next(gen)

# alleno la rete la prima volta, senza validation set per velocizzare il processo
history = model.fit((theta,rho), y, epochs=epochs, batch_size=batch_size)


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 dense (Dense)                  (None, 2)            4           ['input_1[0][0]']                
                                                                                                  
 dense_1 (Dense)                (None, 2)            4           ['input_2[0][0]']                
                                                                                              

In [ ]:
# nella seconda fase di train il numero di dati nel training set e' diminuito
# le epoche sono state ridotte a 50 e il batch size 1024
# in quanto bisogna iniziare a temere overfitting e migliorare invece la precisione

n_train = 1000000

g1,g2 = 10,10
gen = polar_generator(n_train,grid=(g1,g2),noise=0.002,flat=True)

epochs = 50
batch_size = 1024

opt = keras.optimizers.Adam()
model.compile(optimizer=opt, loss='categorical_crossentropy', 
        metrics=['accuracy', my_accuracy]
              )

(theta,rho),y = next(gen)

history = model.fit((theta,rho), y, epochs=epochs, batch_size=batch_size)


Epoch 1/50
977/977 [==============================] - 10s 9ms/step - loss: 0.0607 - accuracy: 0.9775 - my_accuracy: 0.9775
Epoch 2/50
977/977 [==============================] - 9s 10ms/step - loss: 0.0600 - accuracy: 0.9781 - my_accuracy: 0.9781
Epoch 3/50
977/977 [==============================] - 9s 9ms/step - loss: 0.0603 - accuracy: 0.9778 - my_accuracy: 0.9778
Epoch 4/50
977/977 [==============================] - 9s 9ms/step - loss: 0.0600 - accuracy: 0.9779 - my_accuracy: 0.9779
Epoch 5/50
977/977 [==============================] - 9s 10ms/step - loss: 0.0604 - accuracy: 0.9776 - my_accuracy: 0.9776
Epoch 6/50
977/977 [==============================] - 9s 10ms/step - loss: 0.0604 - accuracy: 0.9775 - my_accuracy: 0.9775
Epoch 7/50
977/977 [==============================] - 9s 9ms/step - loss: 0.0605 - accuracy: 0.9776 - my_accuracy: 0.9776
Epoch 8/50
977/977 [==============================] - 9s 9ms/step - loss: 0.0597 - accuracy: 0.9780 - my_accuracy: 0.9780
Epoch 9/50
977/977 [

In [ ]:
# la rete viene nuovamente allenata come nella fase precedente ma per piu' tempo

n_train = 1000000

g1,g2 = 10,10
gen = polar_generator(n_train,grid=(g1,g2),noise=0.002,flat=True)

epochs = 100
batch_size = 1024

opt = keras.optimizers.Adam()
model.compile(optimizer=opt, loss='categorical_crossentropy', 
        metrics=['accuracy', my_accuracy]
              )

(theta,rho),y = next(gen)

history = model.fit((theta,rho), y, epochs=epochs, batch_size=batch_size)


Epoch 1/100
977/977 [==============================] - 10s 10ms/step - loss: 0.0585 - accuracy: 0.9778 - my_accuracy: 0.9778
Epoch 2/100
977/977 [==============================] - 10s 10ms/step - loss: 0.0588 - accuracy: 0.9776 - my_accuracy: 0.9776
Epoch 3/100
977/977 [==============================] - 10s 10ms/step - loss: 0.0584 - accuracy: 0.9778 - my_accuracy: 0.9778
Epoch 4/100
977/977 [==============================] - 11s 11ms/step - loss: 0.0583 - accuracy: 0.9780 - my_accuracy: 0.9780
Epoch 5/100
977/977 [==============================] - 9s 10ms/step - loss: 0.0576 - accuracy: 0.9785 - my_accuracy: 0.9785
Epoch 6/100
977/977 [==============================] - 9s 10ms/step - loss: 0.0588 - accuracy: 0.9777 - my_accuracy: 0.9777
Epoch 7/100
977/977 [==============================] - 10s 10ms/step - loss: 0.0581 - accuracy: 0.9783 - my_accuracy: 0.9783
Epoch 8/100
977/977 [==============================] - 10s 10ms/step - loss: 0.0580 - accuracy: 0.9781 - my_accuracy: 0.9781
Ep

In [ ]:
# la rete attualmente e' gia abbastanza accurata, per evitare overfitting
# ma migliorare comunque la precisione le epoche sono notevolmente ridotte
# e il learning rate abbassato di molto

n_train = 1000000

g1,g2 = 10,10
gen = polar_generator(n_train,grid=(g1,g2),noise=0.002,flat=True)

epochs = 20
batch_size = 1024

opt = keras.optimizers.Adam(learning_rate=0.0000001)
model.compile(optimizer=opt, loss='categorical_crossentropy', 
        metrics=['accuracy', my_accuracy]
              )

(theta,rho),y = next(gen)

history = model.fit((theta,rho), y, epochs=epochs, batch_size=batch_size)


Epoch 1/20
977/977 [==============================] - 10s 10ms/step - loss: 0.0649 - accuracy: 0.9719 - my_accuracy: 0.9719
Epoch 2/20
977/977 [==============================] - 9s 10ms/step - loss: 0.0635 - accuracy: 0.9727 - my_accuracy: 0.9727
Epoch 3/20
977/977 [==============================] - 9s 10ms/step - loss: 0.0623 - accuracy: 0.9734 - my_accuracy: 0.9734
Epoch 4/20
977/977 [==============================] - 9s 10ms/step - loss: 0.0611 - accuracy: 0.9741 - my_accuracy: 0.9741
Epoch 5/20
977/977 [==============================] - 10s 10ms/step - loss: 0.0601 - accuracy: 0.9747 - my_accuracy: 0.9747
Epoch 6/20
977/977 [==============================] - 9s 10ms/step - loss: 0.0591 - accuracy: 0.9752 - my_accuracy: 0.9752
Epoch 7/20
977/977 [==============================] - 10s 10ms/step - loss: 0.0583 - accuracy: 0.9757 - my_accuracy: 0.9757
Epoch 8/20
977/977 [==============================] - 9s 9ms/step - loss: 0.0575 - accuracy: 0.9762 - my_accuracy: 0.9762
Epoch 9/20
977

**Fase di validazione del modello**

In [ ]:
# il modello viene validato su un validation set di 20000 elementi come
# da consegna e un noise di 0.002
# per verificare la qualita' delle predizioni viene effettuata l'operazione
# di evaluate su un set di 20000 per 1000 volte, per ogni ciclo viene verificato
# se l'accuracy risulta sotto il 95%
# il risultato e' stato che con abbastanza frequenza su 1000 cicli di evaluate 
# con validation set da 20000 elementi la percentuale di tentativi con accuracy
# inferiore a 95% e' minore del 10%

# prima verifica con esito
#  7.6 % sotto al 95 %
# 92.4 % sopra al 95 %

g1,g2 = 10,10
batch_size = 256
n_test = 20000
gen = polar_generator(n_test,grid=(g1,g2),noise=0.002,flat=True)

accs = []
mm = 0
tot = 1000
for x in range(tot):
  (theta,rho),y = next(gen)
  score, _, acc = model.evaluate((theta,rho), y, batch_size=batch_size,verbose=0)
  if acc < 0.95:
    mm += 1
    print("Sotto 95% : {}/{}".format(mm, x+1))
  accs.append(acc)


acc = np.mean(accs)
print('Accuracy: {:.1f}%'.format(acc*100))
print('Totali sotto 95% : {}/{}'.format(mm,tot))


Sotto 95% : 1/10
Sotto 95% : 2/23
Sotto 95% : 3/27
Sotto 95% : 4/32
Sotto 95% : 5/63
Sotto 95% : 6/70
Sotto 95% : 7/77
Sotto 95% : 8/93
Sotto 95% : 9/97
Sotto 95% : 10/151
Sotto 95% : 11/161
Sotto 95% : 12/191
Sotto 95% : 13/196
Sotto 95% : 14/216
Sotto 95% : 15/221
Sotto 95% : 16/222
Sotto 95% : 17/233
Sotto 95% : 18/240
Sotto 95% : 19/254
Sotto 95% : 20/256
Sotto 95% : 21/263
Sotto 95% : 22/274
Sotto 95% : 23/311
Sotto 95% : 24/317
Sotto 95% : 25/320
Sotto 95% : 26/321
Sotto 95% : 27/342
Sotto 95% : 28/354
Sotto 95% : 29/380
Sotto 95% : 30/389
Sotto 95% : 31/395
Sotto 95% : 32/398
Sotto 95% : 33/406
Sotto 95% : 34/418
Sotto 95% : 35/440
Sotto 95% : 36/486
Sotto 95% : 37/487
Sotto 95% : 38/490
Sotto 95% : 39/492
Sotto 95% : 40/499
Sotto 95% : 41/507
Sotto 95% : 42/520
Sotto 95% : 43/528
Sotto 95% : 44/549
Sotto 95% : 45/558
Sotto 95% : 46/563
Sotto 95% : 47/568
Sotto 95% : 48/575
Sotto 95% : 49/586
Sotto 95% : 50/587
Sotto 95% : 51/603
Sotto 95% : 52/656
Sotto 95% : 53/657
Sotto 95% :

In [ ]:
# seconda verifica con esito
#  5.9 % sotto al 95 %
# 94.1 % sopra al 95 %

g1,g2 = 10,10
batch_size = 256
n_test = 20000
gen = polar_generator(n_test,grid=(g1,g2),noise=0.002,flat=True)

accs = []
mm = 0
tot = 1000
for x in range(tot):
  (theta,rho),y = next(gen)
  score, _, acc = model.evaluate((theta,rho), y, batch_size=batch_size,verbose=0)
  if acc < 0.95:
    mm += 1
    print("Sotto 95% : {}/{}".format(mm, x+1))
  accs.append(acc)


acc = np.mean(accs)
print('Accuracy: {:.1f}%'.format(acc*100))
print('Totali sotto 95% : {}/{}'.format(mm,tot))


Sotto 95% : 1/1
Sotto 95% : 2/37
Sotto 95% : 3/80
Sotto 95% : 4/99
Sotto 95% : 5/117
Sotto 95% : 6/173
Sotto 95% : 7/177
Sotto 95% : 8/196
Sotto 95% : 9/221
Sotto 95% : 10/223
Sotto 95% : 11/226
Sotto 95% : 12/229
Sotto 95% : 13/243
Sotto 95% : 14/274
Sotto 95% : 15/290
Sotto 95% : 16/297
Sotto 95% : 17/304
Sotto 95% : 18/308
Sotto 95% : 19/313
Sotto 95% : 20/314
Sotto 95% : 21/321
Sotto 95% : 22/339
Sotto 95% : 23/352
Sotto 95% : 24/362
Sotto 95% : 25/381
Sotto 95% : 26/389
Sotto 95% : 27/408
Sotto 95% : 28/411
Sotto 95% : 29/416
Sotto 95% : 30/444
Sotto 95% : 31/468
Sotto 95% : 32/487
Sotto 95% : 33/493
Sotto 95% : 34/522
Sotto 95% : 35/585
Sotto 95% : 36/611
Sotto 95% : 37/612
Sotto 95% : 38/637
Sotto 95% : 39/669
Sotto 95% : 40/691
Sotto 95% : 41/702
Sotto 95% : 42/724
Sotto 95% : 43/731
Sotto 95% : 44/740
Sotto 95% : 45/748
Sotto 95% : 46/774
Sotto 95% : 47/785
Sotto 95% : 48/822
Sotto 95% : 49/831
Sotto 95% : 50/835
Sotto 95% : 51/844
Sotto 95% : 52/848
Sotto 95% : 53/861
Sotto 9

In [ ]:
# terza verifica con esito
#  7.0 % sotto al 95 %
# 93.0 % sopra al 95 %

g1,g2 = 10,10
batch_size = 256
n_test = 20000
gen = polar_generator(n_test,grid=(g1,g2),noise=0.002,flat=True)

accs = []
mm = 0
tot = 1000
for x in range(tot):
  (theta,rho),y = next(gen)
  score, _, acc = model.evaluate((theta,rho), y, batch_size=batch_size,verbose=0)
  if acc < 0.95:
    mm += 1
    print("Sotto 95% : {}/{}".format(mm, x+1))
  accs.append(acc)


acc = np.mean(accs)
print('Accuracy: {:.1f}%'.format(acc*100))
print('Totali sotto 95% : {}/{}'.format(mm,tot))


Sotto 95% : 1/1
Sotto 95% : 2/3
Sotto 95% : 3/6
Sotto 95% : 4/18
Sotto 95% : 5/34
Sotto 95% : 6/44
Sotto 95% : 7/56
Sotto 95% : 8/69
Sotto 95% : 9/74
Sotto 95% : 10/100
Sotto 95% : 11/125
Sotto 95% : 12/126
Sotto 95% : 13/139
Sotto 95% : 14/140
Sotto 95% : 15/189
Sotto 95% : 16/192
Sotto 95% : 17/223
Sotto 95% : 18/259
Sotto 95% : 19/274
Sotto 95% : 20/277
Sotto 95% : 21/290
Sotto 95% : 22/298
Sotto 95% : 23/306
Sotto 95% : 24/309
Sotto 95% : 25/313
Sotto 95% : 26/339
Sotto 95% : 27/344
Sotto 95% : 28/348
Sotto 95% : 29/349
Sotto 95% : 30/354
Sotto 95% : 31/356
Sotto 95% : 32/358
Sotto 95% : 33/418
Sotto 95% : 34/429
Sotto 95% : 35/452
Sotto 95% : 36/483
Sotto 95% : 37/487
Sotto 95% : 38/500
Sotto 95% : 39/522
Sotto 95% : 40/556
Sotto 95% : 41/562
Sotto 95% : 42/578
Sotto 95% : 43/588
Sotto 95% : 44/592
Sotto 95% : 45/633
Sotto 95% : 46/642
Sotto 95% : 47/657
Sotto 95% : 48/664
Sotto 95% : 49/684
Sotto 95% : 50/711
Sotto 95% : 51/744
Sotto 95% : 52/775
Sotto 95% : 53/776
Sotto 95% : 54

In [ ]:
# quarta verifica con esito
#  7.2 % sotto al 95 %
# 92.8 % sopra al 95 %

g1,g2 = 10,10
batch_size = 256
n_test = 20000
gen = polar_generator(n_test,grid=(g1,g2),noise=0.002,flat=True)

accs = []
mm = 0
tot = 1000
for x in range(tot):
  (theta,rho),y = next(gen)
  score, _, acc = model.evaluate((theta,rho), y, batch_size=batch_size,verbose=0)
  if acc < 0.95:
    mm += 1
    print("Sotto 95% : {}/{}".format(mm, x+1))
  accs.append(acc)


acc = np.mean(accs)
print('Accuracy: {:.1f}%'.format(acc*100))
print('Totali sotto 95% : {}/{}'.format(mm,tot))


Sotto 95% : 1/32
Sotto 95% : 2/63
Sotto 95% : 3/67
Sotto 95% : 4/80
Sotto 95% : 5/82
Sotto 95% : 6/120
Sotto 95% : 7/121
Sotto 95% : 8/162
Sotto 95% : 9/184
Sotto 95% : 10/192
Sotto 95% : 11/198
Sotto 95% : 12/207
Sotto 95% : 13/235
Sotto 95% : 14/250
Sotto 95% : 15/295
Sotto 95% : 16/316
Sotto 95% : 17/320
Sotto 95% : 18/327
Sotto 95% : 19/331
Sotto 95% : 20/340
Sotto 95% : 21/344
Sotto 95% : 22/358
Sotto 95% : 23/380
Sotto 95% : 24/384
Sotto 95% : 25/388
Sotto 95% : 26/392
Sotto 95% : 27/461
Sotto 95% : 28/479
Sotto 95% : 29/490
Sotto 95% : 30/510
Sotto 95% : 31/512
Sotto 95% : 32/523
Sotto 95% : 33/527
Sotto 95% : 34/531
Sotto 95% : 35/544
Sotto 95% : 36/556
Sotto 95% : 37/560
Sotto 95% : 38/569
Sotto 95% : 39/584
Sotto 95% : 40/587
Sotto 95% : 41/591
Sotto 95% : 42/604
Sotto 95% : 43/621
Sotto 95% : 44/629
Sotto 95% : 45/635
Sotto 95% : 46/638
Sotto 95% : 47/665
Sotto 95% : 48/713
Sotto 95% : 49/714
Sotto 95% : 50/716
Sotto 95% : 51/725
Sotto 95% : 52/727
Sotto 95% : 53/729
Sotto 9

In [ ]:
# quinta verifica con esito del 6.1 %
#  6.1 % sotto al 95 %
# 93.9 % sopra al 95 %

g1,g2 = 10,10
batch_size = 256
n_test = 20000
gen = polar_generator(n_test,grid=(g1,g2),noise=0.002,flat=True)

accs = []
mm = 0
tot = 1000
for x in range(tot):
  (theta,rho),y = next(gen)
  score, _, acc = model.evaluate((theta,rho), y, batch_size=batch_size,verbose=0)
  if acc < 0.95:
    mm += 1
    print("Sotto 95% : {}/{}".format(mm, x+1))
  accs.append(acc)


acc = np.mean(accs)
print('Accuracy: {:.1f}%'.format(acc*100))
print('Totali sotto 95% : {}/{}'.format(mm,tot))


Sotto 95% : 1/9
Sotto 95% : 2/21
Sotto 95% : 3/33
Sotto 95% : 4/58
Sotto 95% : 5/96
Sotto 95% : 6/111
Sotto 95% : 7/136
Sotto 95% : 8/155
Sotto 95% : 9/182
Sotto 95% : 10/188
Sotto 95% : 11/190
Sotto 95% : 12/193
Sotto 95% : 13/200
Sotto 95% : 14/208
Sotto 95% : 15/251
Sotto 95% : 16/311
Sotto 95% : 17/329
Sotto 95% : 18/343
Sotto 95% : 19/344
Sotto 95% : 20/371
Sotto 95% : 21/379
Sotto 95% : 22/386
Sotto 95% : 23/393
Sotto 95% : 24/394
Sotto 95% : 25/404
Sotto 95% : 26/428
Sotto 95% : 27/445
Sotto 95% : 28/472
Sotto 95% : 29/502
Sotto 95% : 30/533
Sotto 95% : 31/551
Sotto 95% : 32/552
Sotto 95% : 33/554
Sotto 95% : 34/564
Sotto 95% : 35/600
Sotto 95% : 36/644
Sotto 95% : 37/671
Sotto 95% : 38/702
Sotto 95% : 39/719
Sotto 95% : 40/721
Sotto 95% : 41/728
Sotto 95% : 42/735
Sotto 95% : 43/741
Sotto 95% : 44/753
Sotto 95% : 45/782
Sotto 95% : 46/788
Sotto 95% : 47/792
Sotto 95% : 48/805
Sotto 95% : 49/811
Sotto 95% : 50/876
Sotto 95% : 51/912
Sotto 95% : 52/919
Sotto 95% : 53/929
Sotto 95